<a href="https://colab.research.google.com/github/Juhi-Purswani/Fake_News_detection/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
import os
os.chdir('./drive/My Drive/Fake_news_detection')

FileNotFoundError: ignored

In [0]:
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten, Bidirectional, TimeDistributed
from keras.layers import Conv1D, MaxPooling1D, Embedding,  Dropout
from keras.models import Model

from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
import numpy as np
from matplotlib import pyplot as plt
from keras.layers import Dense, Embedding, LSTM, GRU

import string
from nltk.corpus import stopwords
import nltk
import re

In [0]:
data_train=pd.read_csv('data/train2.tsv',delimiter='\t',encoding='utf-8',names=["id", "label", "title", "Coverage","i", "lal", "Ed", "Cerage","d", "lbel", "Egtd", "Covage","Dvds","cds","text"])

In [31]:
data_train.drop(['id','Coverage','i','lal','Ed','Cerage','d','lbel','Egtd','Covage','Dvds','cds'],axis=1,inplace=True)
data_train

,label,title,text
0,false,Says the Annies List political group supports ...,That's a premise that he fails to back up. Ann...
1,half-true,When did the decline of coal start? It started...,"Surovell said the decline of coal ""started whe..."
2,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",Obama said he would have voted against the ame...
3,false,Health care reform legislation is likely to ma...,The release may have a point that Mikulskis co...
4,half-true,The economic turnaround started at the end of ...,"Crist said that the economic ""turnaround start..."
5,true,The Chicago Bears have had more starting quart...,"But Vos specifically used the word ""fired,"" wh..."
6,barely-true,Jim Dunnam has not lived in the district he re...,But determining that would take significant de...
7,half-true,I'm the only person on this stage who has work...,"However, it was not that bill, but another one..."
8,half-true,"However, it took $19.5 million in Oregon Lotte...",But Johnson is correct that many other factors...
9,mostly-true,Says GOP primary opponents Glenn Grothman and ...,Considering that the $532 million figure cover...


In [0]:
data_train.title = data_train.title.str.lower()
data_train.text = data_train.text.str.lower()
data_train.title = data_train.title.str.replace(r'\s\s+',' ')
data_train.text = data_train.text.str.replace(r'\s\s+',' ')
data_train.title = data_train.title.str.replace(r'[^\.\w\s]','') #remove everything but characters and punctuation
data_train.text = data_train.text.str.replace(r'[^\.\w\s]','')
data_train.title = data_train.title.str.replace(r'\.\.+','.') #replace multple periods with a single one
data_train.text = data_train.text.str.replace(r'\.\.+','.')
data_train.title = data_train.title.str.strip() 
data_train.text = data_train.text.str.strip()
#data_train.title = re.sub(r '\d+' , '' , data_train.title)
#data_train.text = re.sub(r '\d+' , '' , data_train.text)
#data_train.title = ''.join([i for i in data_train.title if not i.isdigit()])

data_train.title = data_train.title.str.replace('\d+', '')
data_train.text = data_train.text.str.replace('\d+', '')




In [33]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
stop_words = set(stopwords.words('english'))
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [35]:
stop_words = set(stopwords.words('english'))
data_train.title.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
data_train.text.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

AttributeError: ignored

In [36]:
data_train

,label,title,text
0,false,says the annies list political group supports ...,thats a premise that he fails to back up. anni...
1,half-true,when did the decline of coal start it started ...,surovell said the decline of coal started when...
2,mostly-true,hillary clinton agrees with john mccain by vot...,obama said he would have voted against the ame...
3,false,health care reform legislation is likely to ma...,the release may have a point that mikulskis co...
4,half-true,the economic turnaround started at the end of ...,crist said that the economic turnaround starte...
5,true,the chicago bears have had more starting quart...,but vos specifically used the word fired which...
6,barely-true,jim dunnam has not lived in the district he re...,but determining that would take significant de...
7,half-true,im the only person on this stage who has worke...,however it was not that bill but another one s...
8,half-true,however it took . million in oregon lottery fu...,but johnson is correct that many other factors...
9,mostly-true,says gop primary opponents glenn grothman and ...,considering that the million figure covers ju...


In [37]:
data_train.isnull().sum()

label     0
title     0
text     84
dtype: int64

In [38]:
data_train.dropna()

,label,title,text
0,false,says the annies list political group supports ...,thats a premise that he fails to back up. anni...
1,half-true,when did the decline of coal start it started ...,surovell said the decline of coal started when...
2,mostly-true,hillary clinton agrees with john mccain by vot...,obama said he would have voted against the ame...
3,false,health care reform legislation is likely to ma...,the release may have a point that mikulskis co...
4,half-true,the economic turnaround started at the end of ...,crist said that the economic turnaround starte...
5,true,the chicago bears have had more starting quart...,but vos specifically used the word fired which...
6,barely-true,jim dunnam has not lived in the district he re...,but determining that would take significant de...
7,half-true,im the only person on this stage who has worke...,however it was not that bill but another one s...
8,half-true,however it took . million in oregon lottery fu...,but johnson is correct that many other factors...
9,mostly-true,says gop primary opponents glenn grothman and ...,considering that the million figure covers ju...


In [0]:
texts = []
labels = []

for i in range(data_train.text.shape[0]):
    text1 = data_train.title[i]
    text2 = data_train.text[i]
    text = str(text1) +""+ str(text2)
    texts.append(text)
    labels.append(data_train.label[i])
    
for i in range(len(labels)):
    if(labels[i]=='mostly-true'):
        labels[i] = 'true'
    
    elif(labels[i]=='barely-true'):
        labels[i] = 'false'
    
    elif(labels[i]=='pants-fire'):
        labels[i] = 'false'
    elif(labels[i]=='half-true'):
        labels[i] = 'true'

In [40]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
features_train = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

df = pd.DataFrame(labels) 
labels_train = pd.get_dummies(df)

print('Shape of data tensor:', features_train)
print('Shape of label tensor:', labels_train)

indices = np.arange(features_train.shape[0])
np.random.shuffle(indices)
features_train = features_train[indices]
labels_train = labels_train[indices]

Shape of data tensor: [[    0     0     0 ...     9    24  1642]
 [    0     0     0 ...  1355   431  5307]
 [    0     0     0 ...     6     8   172]
 ...
 [    0     0     0 ...   571   526  1822]
 [    0     0     0 ...    14  1137  1906]
 [    0     0     0 ... 16283     8   112]]
Shape of label tensor:        0_false  0_true
0            1       0
1            0       1
2            0       1
3            1       0
4            0       1
5            0       1
6            1       0
7            0       1
8            0       1
9            0       1
10           0       1
11           0       1
12           1       0
13           0       1
14           1       0
15           0       1
16           0       1
17           1       0
18           0       1
19           0       1
20           1       0
21           0       1
22           0       1
23           0       1
24           1       0
25           1       0
26           0       1
27           0       1
28           1       0
2

KeyError: ignored

In [41]:
#Using Pre-trained word embeddings
GLOVE_DIR = "data" 
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="utf8")
for line in f:
    values = line.split()
    #print(values[1:])
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors in Glove.' % len(embeddings_index))

embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH)

Total 400000 word vectors in Glove.


In [42]:
embedding_vecor_length = 32
modell = Sequential()
modell.add(embedding_layer)
modell.add(Dropout(0.2))
modell.add(Conv1D(filters=32, kernel_size=5, padding='same', activation='relu'))
modell.add(MaxPooling1D(pool_size=2))
modell.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
modell.add(MaxPooling1D(pool_size=2))
modell.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
modell.add(BatchNormalization())
modell.add(Dense(256, activation='relu'))
modell.add(Dense(128, activation='relu'))
modell.add(Dense(64, activation='relu'))
modell.add(Dense(2, activation='softmax'))
modell.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(modell.summary())
modell.fit(features_train, labels_train, epochs=10, batch_size=64)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1000, 100)         2709100   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000, 100)         0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1000, 32)          16032     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 500, 32)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 500, 64)           6208      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 250, 64)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               66000     
__________